In [2]:
# imports
import os
import sys
import pandas as pd
import numpy as np

## Data preparation

I will follow the steps outlined in *"Parameter estimation of the MFD: a maximum likelihood approach"*.  

First, the following cities will be left out:
- Paris (1h intervals, too aggregated)
- Bolton, Birmingham, Groningen, Innsbruck, Manchester, Melbourne, Rotterdam, Torino, Utrecht (partial or no occupancy measurements)

**Step 1: Load raw data and remove selected cities from the two data sets**

In [3]:
# read the data from raw data folder
# we will specify the data types for each column to save memory and prevent errors
dtype_dict = {
    'day': 'str',  # Assuming day is an integer
    'interval': 'int32',  # Assuming interval is an integer
    'detid': 'str',  # Treat detid as a string to avoid mixed types
    'flow': 'float32',  # Assuming flow is a decimal number
    'occ': 'float32',  # Assuming occ is a decimal number
    'error': 'float32',  # Assuming error is a decimal number
    'city': 'category',  # Assuming city names are repeated, use 'category' for memory efficiency
    'speed': 'float32'  # Assuming speed is a decimal number
}

data = pd.read_csv('C:/DTU/Speciale/Kode/Speciale/data/raw/utd19_u.csv', dtype=dtype_dict)

# Parse the day column as a datetime object
data['day'] = pd.to_datetime(data['day'], format='%Y-%m-%d')

detector = pd.read_csv('C:/DTU/Speciale/Kode/Speciale/data/raw/detectors_public.csv')

In [4]:
data

,day,interval,detid,flow,occ,error,city,speed
0,2017-05-06,0,06.X-2li,12.0,0.000000,1.0,augsburg,NaN
1,2017-05-06,300,06.X-2li,12.0,0.000000,1.0,augsburg,NaN
2,2017-05-06,600,06.X-2li,12.0,0.000000,1.0,augsburg,NaN
3,2017-05-06,900,06.X-2li,16.0,0.000000,1.0,augsburg,NaN
4,2017-05-06,1200,06.X-2li,16.0,0.000000,1.0,augsburg,NaN
...,...,...,...,...,...,...,...,...
134380366,2015-11-01,85500,K8D20,120.0,0.015556,0.0,zurich,NaN
134380367,2015-11-01,85680,K8D20,120.0,0.017778,0.0,zurich,NaN
134380368,2015-11-01,85860,K8D20,120.0,0.014444,0.0,zurich,NaN
134380369,2015-11-01,86040,K8D20,200.0,0.028333,0.0,zurich,NaN


In [4]:
detector.head()

,detid,length,pos,fclass,road,limit,citycode,lanes,linkid,long,lat
0,U1-52G,0.196037,0.005512,secondary,Gögginger Straße,50,augsburg,1.0,72.0,10.889553,48.359957
1,U1-51G,0.130039,0.004013,secondary,Gögginger Straße,50,augsburg,1.0,73.0,10.889601,48.359945
2,U1-52L,0.155863,0.022228,secondary,Gögginger Straße,50,augsburg,1.0,70.0,10.889356,48.359876
3,U1-51L,0.197675,0.021889,secondary,Gögginger Straße,50,augsburg,1.0,71.0,10.889396,48.359862
4,U1-62,0.065183,0.024465,secondary,Rosenaustraße,50,augsburg,1.0,68.0,10.889361,48.360578


In [5]:
# Correct spelling of Los Angeles in detector df to match data df
# change from losanageles to losangeles
detector['citycode'] = detector['citycode'].replace('losanageles', 'losangeles')

In [6]:
# # Code for checking if the cities in the two dataframes are the same
# data_cities = data['city'].unique()
# detector_cities = detector['citycode'].unique()
# print(data_cities)
# print(detector_cities)
# # check if the two sets of cities are the same, print diffqerences if not
# if set(data_cities) != set(detector_cities):
#     print('Cities in data but not in detector: ', set(data_cities) - set(detector_cities))
#     print('Cities in detector but not in data: ', set(detector_cities) - set(data_cities))

In [7]:
# We want to remove the following cities from the data
# As the aggregation level is too high 
delete_cities_1 = ['paris']
# or they have partial or no occupancy data
delete_cities_2 = ['birmingham', 'bolton', 'groningen', 'innsbruck', 'manchester', 'melbourne', 'rotterdam', 'torino', 'utrecht']
# present in the detector data but not in the data
delete_cities_3 = ['tokyo']

# Remove the cities from the data
data = data[~data['city'].isin(delete_cities_1 + delete_cities_2 + delete_cities_3)]
detector = detector[~detector['citycode'].isin(delete_cities_1 + delete_cities_2 + delete_cities_3)]

**Step 2: Criteria for discarding observations**

The following type of observations are considereed to be outliers or faulty, and will now be discarded from the data set:
- Values marked as errors in the observations data set
- Occupancy measurements larger than 1
- Flow measurements larger than 2500 veh/hr/ln
- Flow measurements less than 10 veh/hr/ln for occupancy values between 0.2 and 0.75
- Flow measurements higher than 100 veh/hr/ln for occupancy values larger than 0.95 

In [8]:
#######------- We notice that there's a lot of observations with an unclear´
#######------- error destription NaN. We assume they're not errenuous
# Get the number of rows with error values = 1
error_counts = data['error'].value_counts(dropna=False)
print(error_counts)

error
NaN    52673887
1.0    44438436
0.0    29074904
Name: count, dtype: int64


In [9]:
# Remove values marked as errors in the observations data set
# i.e. drop rows with error = 1
data = data[data['error'] != 1]

# Remove occupancy measurements larger than 1
data = data[data['occ'] <= 1]

# Remove flow measurements larger than 2500 veh/hr/ln
data = data[data['flow'] <= 2500]

# Remove flow measurements less than 10 veh/hr/ln for occupancy values between 0.2 and 0.75
data = data[~((data['flow'] < 10) & (data['occ'] > 0.2) & (data['occ'] < 0.75))]

# Remove flow measurements less than 10 veh/hr/ln for occupancy values between 0.2 and 0.75
data = data[~((data['flow'] < 10) & (data['occ'] > 0.2) & (data['occ'] < 0.75))]

# Remove flow measurements higher than 100 veh/hr/ln for occupancy values larger than 0.95
data = data[~((data['flow'] > 100) & (data['occ'] > 0.95))]

**Step 3: Select only the loop detectors with valid data for more than 80% of the time intervals**

In [10]:
# Combine date and interval columns to create unique timestamps
data['timestamp'] = data['day'] + pd.to_timedelta(data['interval'], unit='h')
# Create a new column 'day_interval' combining 'day' and 'interval'
# data['day_interval'] = data['day'].astype(str) + '-' + data['interval'].astype(str).str.zfill(5)
#---- decide which is better

In [11]:
# Select only the loop detectors with valid data for more than 80% of the time intervals

# Create empty list to store the filtered data
filtered_data_list = []
# Create empty list to store statistics for each city
stats_list = []

# Get all the city names
cities = data['city'].unique()

for city in cities:
    # Get the data for the city
    city_data = data[data['city'] == city]

    # Get the number of time intervals in the data
    n_intervals = city_data['timestamp'].nunique()

    # Get the number of valid observations for each detector
    detector_counts = city_data['detid'].value_counts()

    # Get the detectors with more than 80% valid observations
    valid_detectors = detector_counts[detector_counts > 0.8 * n_intervals].index

    # Filter the data for the valid detectors
    city_data = city_data[city_data['detid'].isin(valid_detectors)]

    # Append the filtered data   
    filtered_data_list.append(city_data)

    # Save the number of detectors, intervals, etc. for each city
    total_detectors = len(detector_counts)
    valid_detector_count = len(valid_detectors)
    valid_percentage = valid_detector_count / total_detectors * 100

    # Append the statistics to the list
    stats_list.append({
        'city': city,
        'total_detectors': total_detectors,
        'valid_detectors': valid_detector_count,
        'valid_percentage': valid_percentage,
        'total_intervals': n_intervals,
    })

# Combine all the filtered data into a single DataFrame at once
filtered_data = pd.concat(filtered_data_list, ignore_index=True)

# Create a DataFrame from the statistics list
stats_df = pd.DataFrame(stats_list)

# Check the final filtered DataFrame
# print(filtered_data.info())

In [12]:
print(stats_df)

          city  total_detectors  valid_detectors  valid_percentage  \
0     augsburg              454              106         23.348018   
1        basel               60               50         83.333333   
2         bern              471              294         62.420382   
3     bordeaux              320              254         79.375000   
4       bremen              480              352         73.333333   
5     cagliari               98               96         97.959184   
6    constance               92               77         83.695652   
7    darmstadt              208              127         61.057692   
8        essen               36               36        100.000000   
9    frankfurt               73               73        100.000000   
10        graz              246              187         76.016260   
11     hamburg              325              273         84.000000   
12      kassel              434              367         84.562212   
13      london      

**Step 4: Keep only intervals with valid data for more than 80% of the previously selected loop detectors**

In [13]:
# Select only intervals with valid data for more than 80% of the previously selected loop detectors

# Create empty list to store the filtered data
filtered_data_list = []
# Create empty list to store statistics for each city
city_stats_list = []

# Get all the city names
cities = filtered_data['city'].unique()

for city in cities:
    # Get the data for the city
    city_data = filtered_data[filtered_data['city'] == city]

    # Get the number of time intervals in the data
    n_intervals = city_data['timestamp'].nunique()

    # Get the number of valid observations for each time interval
    interval_counts = city_data['timestamp'].value_counts()

    # Get the intervals with more than 80% valid observations
    valid_intervals = interval_counts[interval_counts > 0.8 * len(city_data['detid'].unique())].index

    # Filter the data for the valid intervals
    city_data = city_data[city_data['timestamp'].isin(valid_intervals)]

    # Append the filtered data
    filtered_data_list.append(city_data)

    # Save the number of intervals, etc. for each city
    total_intervals = len(interval_counts)
    valid_interval_count = len(valid_intervals)
    valid_percentage = valid_interval_count / total_intervals * 100

    # Append the statistics to the list
    city_stats_list.append({
        'city': city,
        'total_intervals': total_intervals,
        'valid_intervals': valid_interval_count,
        'valid_percentage': valid_percentage,
        'total_detectors': len(city_data['detid'].unique()),
    })

# Combine all the filtered data into a single DataFrame at once
filtered_data = pd.concat(filtered_data_list, ignore_index=True)

# Create a DataFrame from the statistics list
city_stats_df = pd.DataFrame(city_stats_list)

# Check the final filtered DataFrame
# print(filtered_data.info())

In [14]:
print(city_stats_df)

          city  total_intervals  valid_intervals  valid_percentage  \
0     augsburg             5757             4605         79.989578   
1        basel             2015             2015        100.000000   
2         bern             2015             1739         86.302730   
3     bordeaux             2016             1689         83.779762   
4       bremen             6720             5760         85.714286   
5     cagliari             7294             7294        100.000000   
6    constance             2016             2016        100.000000   
7    darmstadt             6517             6310         96.823692   
8        essen             6360             6286         98.836478   
9    frankfurt              288              287         99.652778   
10        graz             2880             2515         87.326389   
11     hamburg             7378             7355         99.688262   
12      kassel             1146              975         85.078534   
13      london      

**Step 5: Save a filtered version of the data**

In [15]:
# # Save the filtered data to a CSV file
# filtered_data.to_csv('C:/DTU/Speciale/Kode/Speciale/data/processed/filtered_data.csv', index=False)

**Step 6: Merge the data sets on detector ID**

The two data sets will be merged on city and detid (some detid's repeat between cities).

In [16]:
# Change the name of the column 'citycode' to 'city' in the detector data
detector.rename(columns={'citycode': 'city'}, inplace=True)

# Merge the filtered data with the detector data
merged_data = pd.merge(filtered_data, detector, on=['city', 'detid'], how='inner')

In [17]:
# # Save the merged data to a CSV file
# merged_data.to_csv('C:/DTU/Speciale/Kode/Speciale/data/processed/merged_data.csv', index=False)

In [18]:
merged_data

,day,interval,detid,flow,occ,error,city,speed,timestamp,length,pos,fclass,road,limit,lanes,linkid,long,lat
0,2017-05-06,0,12-1,104.0,0.010000,NaN,augsburg,NaN,2017-05-06 00:00:00,0.264448,0.042628,primary,Berliner Allee,50,1.0,409.0,10.928206,48.365476
1,2017-05-06,300,12-1,104.0,0.010000,NaN,augsburg,NaN,2017-05-18 12:00:00,0.264448,0.042628,primary,Berliner Allee,50,1.0,409.0,10.928206,48.365476
2,2017-05-06,600,12-1,104.0,0.010000,NaN,augsburg,NaN,2017-05-31 00:00:00,0.264448,0.042628,primary,Berliner Allee,50,1.0,409.0,10.928206,48.365476
3,2017-05-06,900,12-1,88.0,0.020000,NaN,augsburg,NaN,2017-06-12 12:00:00,0.264448,0.042628,primary,Berliner Allee,50,1.0,409.0,10.928206,48.365476
4,2017-05-06,1200,12-1,88.0,0.020000,NaN,augsburg,NaN,2017-06-25 00:00:00,0.264448,0.042628,primary,Berliner Allee,50,1.0,409.0,10.928206,48.365476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66998425,2015-11-01,85500,K8D20,120.0,0.015556,0.0,zurich,NaN,2025-08-02 12:00:00,0.235968,0.205066,primary,Seebahnstrasse,50,1.0,199.0,8.521908,47.372198
66998426,2015-11-01,85680,K8D20,120.0,0.017778,0.0,zurich,NaN,2025-08-10 00:00:00,0.235968,0.205066,primary,Seebahnstrasse,50,1.0,199.0,8.521908,47.372198
66998427,2015-11-01,85860,K8D20,120.0,0.014444,0.0,zurich,NaN,2025-08-17 12:00:00,0.235968,0.205066,primary,Seebahnstrasse,50,1.0,199.0,8.521908,47.372198
66998428,2015-11-01,86040,K8D20,200.0,0.028333,0.0,zurich,NaN,2025-08-25 00:00:00,0.235968,0.205066,primary,Seebahnstrasse,50,1.0,199.0,8.521908,47.372198
